In [1]:
import numpy as np
import pandas as pd
import scipy
train_set = pd.read_csv('./dataset/train.csv')#.sample(frac=1.0)[:50000]
test_set = pd.read_csv('./dataset/test.csv')#[:1]
train_set.head(10)

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,房屋朝向,居住状态,卧室数量,厅的数量,卫的数量,出租方式,区,位置,地铁线路,地铁站点,距离,装修情况,月租金
0,1,3072,0.128906,2,0.236364,0.008628,东南,NaN,1,1,1,NaN,11.0,118.0,2.0,40.0,0.764167,NaN,5.602716
1,1,3152,0.132812,1,0.381818,0.017046,东,NaN,1,0,0,NaN,10.0,100.0,4.0,58.0,0.709167,NaN,16.977929
2,1,5575,0.042969,0,0.290909,0.010593,东南,NaN,2,1,2,NaN,12.0,130.0,5.0,37.0,0.572500,NaN,8.998302
3,1,3103,0.085938,2,0.581818,0.019199,南,NaN,3,2,2,NaN,7.0,90.0,2.0,63.0,0.658333,NaN,5.602716
4,1,5182,0.214844,0,0.545455,0.010427,东北,NaN,2,1,1,NaN,3.0,31.0,NaN,NaN,NaN,NaN,7.300509
5,1,1192,0.039062,2,0.309091,0.012579,南,NaN,2,1,1,NaN,3.0,143.0,3.0,59.0,0.495833,NaN,4.923599
6,1,1122,0.125000,0,0.381818,0.010593,南,NaN,3,1,1,NaN,11.0,48.0,2.0,9.0,0.193333,NaN,6.621392
7,1,1251,0.128906,2,0.363636,0.018040,南,NaN,4,2,2,NaN,12.0,52.0,NaN,NaN,NaN,NaN,14.091681
8,1,4718,0.246094,2,0.309091,0.007850,西南,NaN,1,1,1,NaN,3.0,143.0,NaN,NaN,NaN,NaN,4.584041
9,1,2654,0.218750,2,0.890909,0.020026,东南,NaN,2,1,2,NaN,10.0,43.0,4.0,58.0,0.400000,NaN,39.558574


In [2]:
train_set.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 196539 entries, 0 to 196538
Data columns (total 19 columns):
时间          196539 non-null int64
小区名         196539 non-null int64
小区房屋出租数量    195538 non-null float64
楼层          196539 non-null int64
总楼层         196539 non-null float64
房屋面积        196539 non-null float64
房屋朝向        196539 non-null object
居住状态        20138 non-null float64
卧室数量        196539 non-null int64
厅的数量        196539 non-null int64
卫的数量        196539 non-null int64
出租方式        24230 non-null float64
区           196508 non-null float64
位置          196508 non-null float64
地铁线路        91778 non-null float64
地铁站点        91778 non-null float64
距离          91778 non-null float64
装修情况        18492 non-null float64
月租金         196539 non-null float64
dtypes: float64(12), int64(6), object(1)
memory usage: 28.5+ MB


In [3]:
train_set.corr()['月租金'] #相关系数

时间         -0.014485
小区名         0.007568
小区房屋出租数量    0.011822
楼层          0.009495
总楼层         0.139803
房屋面积        0.440718
居住状态       -0.075957
卧室数量        0.315892
厅的数量        0.221741
卫的数量        0.423756
出租方式        0.247789
区           0.146881
位置          0.006541
地铁线路        0.187766
地铁站点        0.019662
距离         -0.075449
装修情况        0.246597
月租金         1.000000
Name: 月租金, dtype: float64

In [4]:
dataset = pd.concat([train_set, test_set]) #是如果没有的 自动补上Nan
# print(train_set.mean())
# print(dataset.mean())

In [5]:
#去掉nan>6： 共474个
nan_series = train_set.isnull().sum(axis=1) > 6
nan_df = pd.DataFrame(nan_series)
nan_index = nan_df.loc[(nan_df[0])==True].index
train_set.drop(nan_index, inplace=True)

In [6]:
cols_oh = ['房屋朝向',#'小区名','区','位置','地铁线路','地铁站点', #其中 区 位置 地铁线路 地铁站点有缺失
          ]
cols_oh_train = cols_oh+['月租金']
cols_oh_test = cols_oh+['id']
cols_fill_0 = ('装修情况')
cols_fill_None = ['居住状态','出租方式','区','位置','地铁线路','地铁站点',] #？？？
cols_fill_mean = ['小区房屋出租数量','距离',
                 ]
# cols_nums = ['时间','小区房屋出租数量','楼层','总楼层','房屋面积','卧室数量','厅的数量','卫的数量','距离','装修情况']

X_train = train_set.drop(['月租金'], axis=1)
y_train = train_set['月租金']
X_test = test_set.drop(['id'], axis=1)

X_train[cols_fill_0].fillna(0, inplace=True) 
X_test[cols_fill_0].fillna(0, inplace=True) 

for col in cols_fill_mean:
#     X_train[col].fillna(train_set[col].mean(), inplace=True)  
#     X_test[col].fillna(train_set[col].mean(), inplace=True) 
    X_train[col].fillna(pd.concat([train_set, test_set])[col].mean(), inplace=True)   #加上测试集里的数据一起算Mean
    X_test[col].fillna(pd.concat([train_set, test_set])[col].mean(), inplace=True) 
#     d1.median() #中位数 d1.mode() #众数 d1.var() #方差 d1.std() 标准差
for col in cols_fill_None:
    X_train[col].fillna('0', inplace=True)  
    X_test[col].fillna('0', inplace=True)  

X_train_drop_oh = X_train.drop(cols_oh, axis=1)
X_test_drop_oh = X_test.drop(cols_oh, axis=1)

X_train_drop_oh.shape, X_test_drop_oh.shape

((196065, 17), (56279, 17))

In [7]:
# a = (train_set.isnull().sum(axis=1)>6)
# b=pd.DataFrame(a)
# c=b.loc[(b[0])==True].index
# len(b.drop(c)), len(b)

In [8]:
X_train_drop_oh.head(10)

,时间,小区名,小区房屋出租数量,楼层,总楼层,房屋面积,居住状态,卧室数量,厅的数量,卫的数量,出租方式,区,位置,地铁线路,地铁站点,距离,装修情况
0,1,3072,0.128906,2,0.236364,0.008628,0,1,1,1,0,11,118,2,40,0.764167,0.0
1,1,3152,0.132812,1,0.381818,0.017046,0,1,0,0,0,10,100,4,58,0.709167,0.0
2,1,5575,0.042969,0,0.290909,0.010593,0,2,1,2,0,12,130,5,37,0.572500,0.0
3,1,3103,0.085938,2,0.581818,0.019199,0,3,2,2,0,7,90,2,63,0.658333,0.0
4,1,5182,0.214844,0,0.545455,0.010427,0,2,1,1,0,3,31,0,0,0.550350,0.0
5,1,1192,0.039062,2,0.309091,0.012579,0,2,1,1,0,3,143,3,59,0.495833,0.0
6,1,1122,0.125000,0,0.381818,0.010593,0,3,1,1,0,11,48,2,9,0.193333,0.0
7,1,1251,0.128906,2,0.363636,0.018040,0,4,2,2,0,12,52,0,0,0.550350,0.0
8,1,4718,0.246094,2,0.309091,0.007850,0,1,1,1,0,3,143,0,0,0.550350,0.0
9,1,2654,0.218750,2,0.890909,0.020026,0,2,1,2,0,10,43,4,58,0.400000,0.0


In [9]:
"""
print(reg.feature_importances_[:20])
# RF50 f0.6
[1.76440031e-03 2.57988045e-02 1.13015335e-02 6.79202866e-02
 3.81498157e-01 2.77022299e-03 2.47302186e-02 8.82714002e-03
 6.79550954e-02 1.63551114e-03 2.02146844e-02 4.88915164e-03
 1.38254925e-02 0.00000000e+00 2.46905469e-07 1.09519720e-07
 1.36155775e-05 1.60649552e-06 4.55505150e-06 1.82883517e-06]
"""

'\nprint(reg.feature_importances_[:20])\n# RF50 f0.6\n[1.76440031e-03 2.57988045e-02 1.13015335e-02 6.79202866e-02\n 3.81498157e-01 2.77022299e-03 2.47302186e-02 8.82714002e-03\n 6.79550954e-02 1.63551114e-03 2.02146844e-02 4.88915164e-03\n 1.38254925e-02 0.00000000e+00 2.46905469e-07 1.09519720e-07\n 1.36155775e-05 1.60649552e-06 4.55505150e-06 1.82883517e-06]\n'

In [10]:
from sklearn.preprocessing import LabelEncoder,OneHotEncoder,LabelBinarizer #要先转化为数值，再onehot
def get_oh(cols): #输入需要onehot的list
#     global X_train_oh, X_test_oh
    X_train_oh = np.arange(X_train.shape[0]).reshape(X_train.shape[0],1)
    X_test_oh = np.arange(X_test.shape[0]).reshape(X_test.shape[0],1)
    
    for col in cols:     
        
        col_train = X_train[col].astype('str')
        col_test = X_test[col].astype('str')
        
        oh_fea = LabelBinarizer().fit(col_train) #去onehot特征
        train_oh = oh_fea.transform(col_train)
        test_oh = oh_fea.transform(col_test)        
        print(col,'：',train_oh.shape, test_oh.shape)
        
        X_train_oh = np.append(X_train_oh, train_oh, axis=1)
        X_test_oh = np.append(X_test_oh, test_oh, axis=1)
    
    np.delete(X_train_oh, 0, axis=1) #因为刚初始化时多了一个arange
    np.delete(X_test_oh, 0, axis=1)
    return X_train_oh, X_test_oh

X_train_oh, X_test_oh = get_oh(cols_oh[:])

# 这是先先数值化再onehot的
# direct_train_labelencoded = LabelEncoder().fit_transform(direct_train) #数值化
# print(direct_train_labelencoded, direct_train_labelencoded.shape)
# oh = OneHotEncoder(sparse=False).fit(direct_train_labelencoded.reshape(-1,1)) #要转化为(xxx,1)
# direct_oh = oh.transform(direct_train_labelencoded.reshape(-1,1)) 

# direct_train_oh, direct_test_oh = get_oh('房屋朝向')
# name_train_oh, name_test_oh = get_oh('小区名')

房屋朝向 ： (196065, 64) (56279, 64)


In [11]:
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

# 归一化
# mms = MinMaxScaler((0,1))
# mms.fit(X_train_drop_oh)
# X_train_drop_oh = mms.transform(X_train_drop_oh)
# X_test_drop_oh = mms.transform(X_test_drop_oh)


#标准化
ss = StandardScaler()
ss.fit(X_train_drop_oh)
X_train_drop_oh = ss.transform(X_train_drop_oh)
X_test_drop_oh = ss.transform(X_test_drop_oh)


'''
X_train_mms = mms.transform(X_train[cols_nums])
X_test_mms = mms.transform(X_test[cols_nums])

# 替换为归一化之后的属性值
X_train[cols_nums] = X_train_mms[cols_nums]
X_test[cols_nums] = X_tesr_mms[cols_nums]
'''
print()

In [12]:
#小区名太多。。。单独处理
# X_train_oh_name, X_test_oh_name = get_oh([cols_oh[0]]) #因为只有一个值，要加上括号

# from sklearn.decomposition import PCA
# pca = PCA(n_components=1000)
# X_train_oh_name

In [13]:
# dir_index = direct_train.value_counts().index
# list(dir_index)
X_train.shape, X_train_oh.shape, X_test.shape, X_test_oh.shape

((196065, 18), (196065, 65), (56279, 18), (56279, 65))

In [14]:
#合并两个 X
X_train = np.append(X_train_drop_oh, X_train_oh, axis=1)
X_test = np.append(X_test_drop_oh, X_test_oh, axis=1)
# 5924

#合并多个 
# X_train = np.hstack((X_train, direct_train_oh, name_train_oh)) #横向合并
# X_test = np.hstack((X_test, direct_test_oh, name_test_oh)) #横向合并
X_train.shape, X_test.shape, 

((196065, 82), (56279, 82))

In [15]:
# 整体降维
# from sklearn.decomposition import PCA
# import time
# start_pca = time.time()
# pca = PCA(n_components=5900)
# pca.fit(X_train)
# X_train = pca.transform(X_train)
# X_test = pca.transform(X_test)
# print('时间', time.time() - start_pca)
# X_train.shape, X_test.shape

In [16]:
# 划分数据，调参
from sklearn.model_selection import train_test_split
X_train2, X_test2, y_train2, y_test2 = train_test_split(X_train, y_train, test_size=0.2, random_state=88)

### XGB：

In [17]:
# from sklearn.kernel_ridge import KernelRidge
# from sklearn import svm
# from sklearn.ensemble import RandomForestRegressor
# from xgboost import XGBRegressor
# from sklearn.model_selection import cross_val_score
# from sklearn.metrics import mean_squared_error
# from sklearn.grid_search import GridSearchCV
# from sklearn.grid_search import RandomizedSearchCV
# from sklearn.externals import joblib
# import time
# start = time.time()

# # lasso = Lasso(alpha=0.0005,max_iter=10000)
# # ridge = Ridge(alpha=60)
# # svr = svm.SVR(C=1,kernel='rbf',gamma='auto',epsilon=0.1)
# # ker = KernelRidge(alpha=20 ,kernel='polynomial',degree=3 , coef0=0.8)
# # ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
# # bay = BayesianRidge()
# # xgb = XGBRegressor()
# lsvr = svm.LinearSVR(C=1)
# rf = RandomForestRegressor(n_estimators=300, max_features=0.55, max_depth=None,
#                            min_samples_split=2, verbose=1, n_jobs=-1,) #服务器10:122s; 100:665s 200:957s
# # n为20时:0.39/2.23；40时0.34,2.15；50时0.35/2.19。特征0.99和n20的数据差不多，0.8时0.4/2.27
# xgb = XGBRegressor(n_estimators=15, reg_alpha=0, reg_lambda=1, learning_rate=0.2, 
#                    min_child_weight=1, max_depth=40, silent=False, n_jobs=40)  

# reg = xgb
# joblib.dump(reg,'model.pkl')
# # reg.fit(X_train2, y_train2, )

# # param_svr = {'kernel':['rbf', 'poly','linear'],
# #              'C':scipy.stats.expon(scale=1), #指数分布，mean≈std≈1             
# #             }

# # param_rf = {#'n_estimators':[20,50,100], #100>50>20>10
# #         'max_features':[0.6,0.55],#0.5>0.45&0.55>0.5       0.6>0.7>0.8>0.9>auto≈0.99;0.7>sqrt&log2*0.5
# # #         'max_depth':[70,100], #None>100≈70>50>20>10
# # #         'min_samples_split':[2,5] #2>5>10>20>50
# # #         'min_samples_leaf':[1,10] #1>10
# #        }  
# # param_xgb = {
# #         'n_estimators':[50,100], 
# #         'max_depth':[15, 20], 
# # #         'learning_rate':[0.1,0.2], #0.2>0.15&0.3
# # #         'reg_lambda':[1, 1.2]
# #        } 
# # gridsearch = GridSearchCV(reg, param_grid=param_xgb, scoring='neg_mean_squared_error', iid=False, cv=4, n_jobs=4)  
# # gridsearch.fit(X_train,y_train)  
# # print(gridsearch.grid_scores_, gridsearch.best_params_, gridsearch.best_score_, gridsearch.best_estimator_)

# # randsearch = RandomizedSearchCV(reg, param_distributions=param_svr, scoring='neg_mean_squared_error', cv=3, n_iter=10, n_jobs=-1)
# # randsearch.fit(X_train,y_train)  
# # print(randsearch.best_params_, randsearch.best_score_)

# # 交叉验证
# # score = cross_val_score(reg, X_train2, y_train2, cv=3,  n_jobs=-1)
# # print(score)

# # reg = joblib.load('model.pkl')
# mse_train2 = mean_squared_error(y_train2, reg.predict(X_train2))
# mse_test2 = mean_squared_error(y_test2, reg.predict(X_test2))
# print(mse_train2, mse_test2)
# print(reg)
# print('时间：',time.time() - start)

random_state=42,lr=0.2,reg_lambda=1 时   
n10/dep50 ：2.17/4.41(reg_lambda=1.2时：2.32/4.48, reg_alpha=0.2时：2.24/4.37)  
n15/dep40 ：0.46/2.73
n20/dep20 ：0.77/2.77  
n20/dep40 ：0.12/2.40(改中位数，缩放到0,2：差不多； lr=0.15时：0.50/2.74)  
n20/dep50 ：0.10/2.38  
n30/dep30 ：0.10/2.34(改中位数，缩放到0,2：0.10/2.29）
n30/dep50 ：0.01/2.28  
n30/dep80 ：0.007/2.29  
n40/dep40 ：0.087/2.32  
n40/dep50 ：0.003/2.27  (min_child_weight=3时：线上2.32)
n80/dep25 ：0.27/2.46(min_child_weight=3； =5时：0.40/2.46 线上2.33；=7时：0.60/2.60；都8min)

n100/dep15 ：1.07/2.90
n100/dep20 ：0.466/2.49 (min_child_weight=3时：0.56/2.72 683s)
n200/dep10 ：4.32/5.38
n200/dep15 ：0.99/2.89(child=3， 802s)
n300/dep10 ：1.81/3.345 826s
n400/dep3  ：6.24/6.99

### RF：

In [41]:
from sklearn.kernel_ridge import KernelRidge
from sklearn import svm
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_squared_error
from sklearn.grid_search import GridSearchCV
from sklearn.grid_search import RandomizedSearchCV
from sklearn.externals import joblib
import time
start = time.time()

# lasso = Lasso(alpha=0.0005,max_iter=10000)
# ridge = Ridge(alpha=60)
# svr = svm.SVR(C=1,kernel='rbf',gamma='auto',epsilon=0.1)
# ker = KernelRidge(alpha=20 ,kernel='polynomial',degree=3 , coef0=0.8)
# ela = ElasticNet(alpha=0.005,l1_ratio=0.08,max_iter=10000)
# bay = BayesianRidge()
# xgb = XGBRegressor()
lsvr = svm.LinearSVR(C=1)
rf = RandomForestRegressor(n_estimators=500, max_features=0.5, max_depth=None,
                           min_samples_split=2, min_samples_leaf=1,
                           oob_score=False, verbose=0, n_jobs=-1,) #服务器10:122s; 100:665s 200:957s
# n为20时:0.39/2.23；40时0.34,2.15；50时0.35/2.19。特征0.99和n20的数据差不多，0.8时0.4/2.27
xgb = XGBRegressor(n_estimators=15, reg_alpha=0, reg_lambda=1, learning_rate=0.2, 
                   min_child_weight=1, max_depth=40, silent=False, n_jobs=40)  

reg = rf
reg.fit(X_train, y_train, )
joblib.dump(reg,'1152_dropna_ss_rf500_f05.pkl')
reg.fit(X_train2, y_train2, )

# param_svr = {'kernel':['rbf', 'poly','linear'],
#              'C':scipy.stats.expon(scale=1), #指数分布，mean≈std≈1             
#             }

# param_rf = {#'n_estimators':[100,500,800], #100>50>20>10
# #         'max_features':[0.48,0.52,0.5],#0.5>0.45&0.55>0.5       0.6>0.7>0.8>0.9>auto≈0.99;0.7>sqrt&log2*0.5
# #         'max_depth':[None,20], #None>100≈70>50>20>10
#         'min_samples_split':[2,5], #2>5>10>20>50
#         'min_samples_leaf':[1,5] #1>10
#        }  
# param_xgb = {
#         'n_estimators':[50,100], 
#         'max_depth':[15, 20], 
# #         'learning_rate':[0.1,0.2], #0.2>0.15&0.3
# #         'reg_lambda':[1, 1.2]
#        } 
# gridsearch = GridSearchCV(reg, param_grid=param_rf, scoring='neg_mean_squared_error',
#                           iid=False, cv=4, n_jobs=-1)  
# gridsearch.fit(X_train,y_train)  
# print(gridsearch.grid_scores_,'\n\n',gridsearch.best_params_, gridsearch.best_score_, gridsearch.best_estimator_)

# randsearch = RandomizedSearchCV(reg, param_distributions=param_svr, scoring='neg_mean_squared_error', cv=3, n_iter=10, n_jobs=-1)
# randsearch.fit(X_train,y_train)  
# print(randsearch.best_params_, randsearch.best_score_)

# 交叉验证
# score = cross_val_score(reg, X_train2, y_train2, cv=3,  n_jobs=-1)
# print(score)

# reg = joblib.load('model.pkl')
mse_train2 = mean_squared_error(y_train2, reg.predict(X_train2))
mse_test2 = mean_squared_error(y_test2, reg.predict(X_test2))
print(mse_train2, mse_test2)
print(reg)
print('时间：',time.time() - start)

0.2877316398533023 2.2034090674529976
RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
           max_features=0.5, max_leaf_nodes=None,
           min_impurity_decrease=0.0, min_impurity_split=None,
           min_samples_leaf=1, min_samples_split=2,
           min_weight_fraction_leaf=0.0, n_estimators=500, n_jobs=-1,
           oob_score=False, random_state=None, verbose=0, warm_start=False)
时间： 68.90750026702881


RandomForest的 (0,1)+mean (random_state=42) ：  
f=0.8  
n50:0.30/2.20(去掉nan+把房间变0后：0.312/2.11)
n20:0.35/2.35

f=0.6
n50:0.31/2.215 (去掉nan后，0.315/2.176，再把房间变0后：0.316/2.132；再选择标准化后：0.313/2.115)
n50:0.30/2.20  (dep=250)
n100:0.287/2.16(6152_dropna_standardscaler后0.295/2.087。。提交是2.14)
n100:0.289/2.34(dep=200)
n100:0.547/2.347(splits=5)
n200提交2.13 

f=0.55  
n50:0.30/2.225  
n100:0.289/2.17
n200:0.290/2.131

(0,2)+median+f0.55：  
200:0.282/2.167    
300:0.282/2.116

(random_state=42) ：  
n200：0.278/2.096 (6152_dropna_standardscaler后)

In [19]:
# 用最优参数fit

# reg = RandomForestRegressor(n_estimators=50, max_depth=15, min_samples_split=50, n_jobs=-1)
# reg.fit(X_train2, y_train2, )
# mse_train2 = mean_squared_error(y_train2, reg.predict(X_train2))
# mse_test2 = mean_squared_error(y_test2, reg.predict(X_test2))
# print(mse_train2, mse_test2)

In [42]:
reg = joblib.load('1152_dropna_ss_rf500_f05.pkl')
y_pred = reg.predict(X_test)
y_pred.shape

(56279,)

In [43]:
with open('1152_dropna_ss_rf500_f05.csv', 'w') as f:
    id = test_set['id']
    f.write('id,price'+'\n')
    for i in range(len(y_pred)):
        f.write(str(id[i])+','+str(y_pred[i])+'\n')

# keras

In [22]:
# from keras import models
# from keras import layers
# from keras import optimizers

# network = models.Sequential()
# network.add(layers.Dense(128, activation='relu', input_shape=(X_train2.shape[1], )))
# # network.add(layers.Dense(128, activation='relu'))
# network.add(layers.Dense(64, activation='sigmoid'))
# network.add(layers.Dense(1))
# network.compile(optimizer=optimizers.RMSprop(lr=0.0001),
#                loss='mse',
#                metrics=['mse'])

# network.fit(X_train2, y_train2, batch_size=256, epochs=3)

# train_mse =network.evaluate(X_train2, y_train2)
# test_mse =network.evaluate(X_test2, y_test2)
# print(train_mse, test_mse)